In [1]:
!pip install azure-search-documents --pre

     |████████████████████████████████| 233 kB 22.2 MB/s eta 0:00:01


In [82]:
 import os
 from azure.core.credentials import AzureKeyCredential
 from azure.search.documents.indexes import SearchIndexClient 
 from azure.search.documents import SearchClient
 from azure.search.documents.indexes.models import (
     ComplexField,
     CorsOptions,
     SearchIndex,
     ScoringProfile,
     SearchFieldDataType,
     SimpleField,
     SearchableField
 )

In [103]:
# Setup the endpoint
endpoint = 'https://team6search.search.windows.net/'
headers = {'Content-Type': 'application/json',
           'api-key': 'F2DCB42DF1164E3CEC7FB79E5BE7DEA5'}
params = {
    'api-version': '2020-06-30'
}

In [104]:
# Create an index
index_name = "indexteam6"
index_payload = {
    "name": index_name,
    "fields": [
        {
            "name": "id",
            "type": "Edm.String",
            "key": "true",
            "searchable": "true",
            "filterable": "true",
            "facetable": "false",
            "sortable": "true"
        },
        {
            "name": "metadata_storage_path",
            "type": "Edm.String",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "metadata_storage_name",
            "type": "Edm.String",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "content",
            "type": "Edm.String",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "metadata_storage_size",
            "type": "Edm.String",
            "searchable": "false",
            "filterable": "false",
            "facetable": "false"
        }
        ,
        {
            "name": "metadata_storage_last_modified",
            "type": "Edm.String",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false"
        }
    ]
}

r = requests.put(endpoint + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)

201


In [46]:
import json
import requests

# Create a data source
datasourceConnectionString = "DefaultEndpointsProtocol=https;AccountName=hackerteam6;AccountKey=/4ppEzFcA0g41yfqB171ya1MVx2JcZPXq6UKxiIubVym2vWbHrLdl9Wx/vlmFqjzeu9PPGZuZ4Ldop7bOEtvuA==;EndpointSuffix=core.windows.net"
datasource_name = "files"
datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities.",
    "type": "azureblob",
    "credentials": {
        "connectionString": datasourceConnectionString
    },
    "container": {
        "name": "file"
    }
}
r = requests.put(endpoint + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)

201


In [105]:
# Create an indexer
indexer_name = "indexer6"
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "fieldMappings": [
        {
            "sourceFieldName": "metadata_storage_path",
            "targetFieldName": "id",
            "mappingFunction":
            {"name": "base64Encode"},
        },
        {
            "sourceFieldName": "metadata_storage_path",
            "targetFieldName": "metadata_storage_path"
        }
    ],
    "parameters":
    {
        "maxFailedItems": -1,
        "maxFailedItemsPerBatch": -1,
        "configuration":
        {
            "dataToExtract": "contentAndMetadata"
        }
    }
}

r = requests.post("https://team6search.search.windows.net/indexers",
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)

201


In [85]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

# We'll connect to the Azure Cognitive Search public sandbox and send a
# query to its "nycjobs" index built from a public dataset of available jobs
# in New York.
api_key = "F2DCB42DF1164E3CEC7FB79E5BE7DEA5"

# Create a SearchClient to send queries
endpoint = "https://team6search.search.windows.net/indexes/indexteam6?api-version=2020-06-30&search=*"
credential = AzureKeyCredential(api_key)
client = SearchClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=credential)

In [106]:
results = client.search(search_text="\"New York\"", include_total_count=True)

print ('Total Documents Matching Query: \n', results.get_count())
print("======================================================================")

for result in results:
    print("New File")
    print("======================================================================")
    print("ID: {} \n URL: {} \n File Name: {} \n Content: {} \n Size: {} \n Last Modified: {} \n\n".format(result["id"], result["metadata_storage_path"],result["metadata_storage_name"],result["content"],result["metadata_storage_size"],result["metadata_storage_last_modified"]))


Total Documents Matching Query: 
 18
New File
ID: aHR0cHM6Ly9oYWNrZXJ0ZWFtNi5ibG9iLmNvcmUud2luZG93cy5uZXQvZmlsZS9jb2xsYXRlcmFsL05ldyUyMFlvcmslMjBCcm9jaHVyZS5wZGY1 
 URL: https://hackerteam6.blob.core.windows.net/file/collateral/New%20York%20Brochure.pdf 
 File Name: New York Brochure.pdf 
 Content: 
Margie’s Travel Presents… 

New York 
The City of New York, often called New York City or simply 

New York, is the most populous city in the United States. 

With an estimated 2015 population of 8,550,405 distributed 

over a land area of about 302.6 square miles, New York 

City is also the most densely populated major city in the 

United States. Located at the southern tip of the state of 

New York, the city is the center of the New York 

metropolitan area, one of the most populous urban 

agglomerations in the world. A global power city, New York 

City exerts a significant impact upon commerce, finance, 

media, art, fashion, research, technology, education, and 

entertainment, its